<a href="https://colab.research.google.com/github/askot19/Analise_de_dados/blob/main/Telefonia/3%20-%20Modelos%20Preditivos/3%20-%20Modelos%20Preditivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align = 'left'>
<img src = 'https://i.imgur.com/8LTNXxF.jpg'>
</p>

---
## **Data Scientist & Analyst**

*by [Paulo Henrique de Melo Ferreira](https://www.linkedin.com/in/paulo-henrique-ferreira-20151374/)*

---

<p style="text-align: justify;">
O atual projeto visa analisar dados da <b>Alura Voz</b>, a qual consiste em uma empresa de telecomunicações. O objetivo desta análise é obter insights acerca do <b>Churn Rate</b> da empresa e encontrar fatores que estejam influenciando essa métrica.
Os dados desta análise foram obtidos através de uma <b>API</b> por webscrapping.

Na primeira etapa os dados foram trabalhandos, montados e limpos para que aqui fosse possível se realizar a análise e entendimento, principalmente do <b> Churn Rate</b>.

Na segunda etapa foram construídos gráficos no intuito de se entender o comportamento dos dados, se há alguma influência do gênero dos clientes, presença de dependentes ou algum outro fator associado. Ao final, foi realizada uma visualização das correlações entre as variáveis.

Nesta terceira e última etapa, faremos a construção de <b>Modelos Preditivos</b>, a fim de prever a evasão futura dos atuais clientes que compõem a estrutura da empresa. Para isso, realizaremos o método <b>One Hot Encoding (OHE)</b>, pois os algoritmos de Machine Learning não são capazes de trabalhar com textos, sendo assim precisamos converter as variáveis categóricas para uma grandeza capaz de ser analizada pelo modelo.

Estuda-se ainda a possibilidade de aplicarmos algum método de conferência de peso das variáveis como o Weight of Evidence (WOE).

Portando as principais características, podemos preparar os dados para treinamento e predição por meio de modelos estatísticos, dentre os quais serão utilizados a <b> Regressão Logística </b> e o modelo de <b> Random Forest </b>.

In [58]:
# Importação de pacotes para análise de dados
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif

In [59]:
# Importando dados após a limpeza
dados = pd.read_csv('https://raw.githubusercontent.com/askot19/Projetos/main/Telefonia/Dados/aluravoz_clean.csv')
dados.head()

,ID,Evasao,Genero,Senior,Casado(a),Dependentes,Posse,ServicoTelefonico,MultiplasLinhas,ServicodeInternet,...,BackupOnline,ProtecaodeDispositivo,SuporteTecnico,StreamingTV,StreamingFilmes,Contrato,CobrancasemPapel,MetododePagamento,CobrancasMensais,CobrancasTotais
0,0002-ORFBO,Nao,Mulher,0,Sim,Sim,9,Sim,Nao,DSL,...,Sim,Nao,Sim,Sim,Nao,Anual,Sim,Correspondencia,65.6,593.30
1,0003-MKNFE,Nao,Homem,0,Nao,Nao,9,Sim,Sim,DSL,...,Nao,Nao,Nao,Nao,Sim,Mensal,Nao,Correspondencia,59.9,542.40
2,0004-TLHLJ,Sim,Homem,0,Nao,Nao,4,Sim,Nao,Fibra optica,...,Nao,Sim,Nao,Nao,Nao,Mensal,Sim,Eletronico,73.9,280.85
3,0011-IGKFF,Sim,Homem,1,Sim,Nao,13,Sim,Nao,Fibra optica,...,Sim,Sim,Nao,Sim,Sim,Mensal,Sim,Eletronico,98.0,1237.85
4,0013-EXCHZ,Sim,Mulher,1,Sim,Nao,3,Sim,Nao,Fibra optica,...,Nao,Nao,Sim,Sim,Nao,Mensal,Sim,Correspondencia,83.9,267.40


Para trabalharmos com modelos estatísticos, não podemos simplesmente utilizar variáveis categóricas, pois os algoritmos somente entendem valores numéricos. Dessa maneira precisamos realizar um tratamento das variáveis categóricas de interesse.

Como vimos anteriormente, a característica gênero não influencia na tomada de decisão entre permanecer ou não com os serviços apresentados pela Alura Voz, entretanto, mesmo que houvesse alguma diferença entre as categorias, não utilizaríamos essa característica a fim de evitar viéses equivocados acerca desses dados.

As demais categorias não apresentam possiblidade de enviesamento, então podemos analisar todas essas e avaliar o seu peso de contribuição para a variável de resposta.

Para a conversão das variáveis categóricas vamos utilizar o médoto **One Hot Encoding**. Que atribui cada categoria como uma coluna de um **Data Frame**, cada linha que este valor aparece no Data Frame original, ele recebe o valor 1 e, caso não apareça, recebe o valor 0. Dessa maneira pode-se considerar como o efeito de uma variável afeta a variável de resposta, sem atribuir uma ponderação à estatística da distribuição desses dados.

In [60]:
#Conferindo colunas
dados.columns

Index(['ID', 'Evasao', 'Genero', 'Senior', 'Casado(a)', 'Dependentes', 'Posse',
       'ServicoTelefonico', 'MultiplasLinhas', 'ServicodeInternet',
       'SegurancaOnline', 'BackupOnline', 'ProtecaodeDispositivo',
       'SuporteTecnico', 'StreamingTV', 'StreamingFilmes', 'Contrato',
       'CobrancasemPapel', 'MetododePagamento', 'CobrancasMensais',
       'CobrancasTotais'],
      dtype='object')

In [61]:
# Coletando colunas importantes para análise
features = ['MultiplasLinhas' ,'ServicodeInternet', 'SegurancaOnline','BackupOnline', 'ProtecaodeDispositivo', 'SuporteTecnico', 'StreamingTV', 'StreamingFilmes', 'Contrato', 'MetododePagamento']

In [62]:
# Construindo um novo data frame para executar as predições
df_ml = dados[features].copy()
df_ml.head()

,MultiplasLinhas,ServicodeInternet,SegurancaOnline,BackupOnline,ProtecaodeDispositivo,SuporteTecnico,StreamingTV,StreamingFilmes,Contrato,MetododePagamento
0,Nao,DSL,Nao,Sim,Nao,Sim,Sim,Nao,Anual,Correspondencia
1,Sim,DSL,Nao,Nao,Nao,Nao,Nao,Sim,Mensal,Correspondencia
2,Nao,Fibra optica,Nao,Nao,Sim,Nao,Nao,Nao,Mensal,Eletronico
3,Nao,Fibra optica,Nao,Sim,Sim,Nao,Sim,Sim,Mensal,Eletronico
4,Nao,Fibra optica,Nao,Nao,Nao,Sim,Sim,Nao,Mensal,Correspondencia


In [63]:
# Construindo One Hot Encoding
df_ohe = pd.get_dummies(df_ml)
df_ohe.head()

,MultiplasLinhas_Nao,MultiplasLinhas_Sem servico telefonico,MultiplasLinhas_Sim,ServicodeInternet_DSL,ServicodeInternet_Fibra optica,ServicodeInternet_Nao,SegurancaOnline_Nao,SegurancaOnline_Sem servico de Internet,SegurancaOnline_Sim,BackupOnline_Nao,...,StreamingFilmes_Nao,StreamingFilmes_Sem servico de Internet,StreamingFilmes_Sim,Contrato_Anual,Contrato_Dois anos,Contrato_Mensal,MetododePagamento_Cartao de credito,MetododePagamento_Correspondencia,MetododePagamento_Debito automatico,MetododePagamento_Eletronico
0,1,0,0,1,0,0,1,0,0,0,...,1,0,0,1,0,0,0,1,0,0
1,0,0,1,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,1,0,0
2,1,0,0,0,1,0,1,0,0,1,...,1,0,0,0,0,1,0,0,0,1
3,1,0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1
4,1,0,0,0,1,0,1,0,0,1,...,1,0,0,0,0,1,0,1,0,0


In [64]:
# Criando coluna de testes
df_ohe['ID'] = 'none'

In [65]:
# Verificando Data Frame
df_ohe.head()

,MultiplasLinhas_Nao,MultiplasLinhas_Sem servico telefonico,MultiplasLinhas_Sim,ServicodeInternet_DSL,ServicodeInternet_Fibra optica,ServicodeInternet_Nao,SegurancaOnline_Nao,SegurancaOnline_Sem servico de Internet,SegurancaOnline_Sim,BackupOnline_Nao,...,StreamingFilmes_Sem servico de Internet,StreamingFilmes_Sim,Contrato_Anual,Contrato_Dois anos,Contrato_Mensal,MetododePagamento_Cartao de credito,MetododePagamento_Correspondencia,MetododePagamento_Debito automatico,MetododePagamento_Eletronico,ID
0,1,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,none
1,0,0,1,1,0,0,1,0,0,1,...,0,1,0,0,1,0,1,0,0,none
2,1,0,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,none
3,1,0,0,0,1,0,1,0,0,0,...,0,1,0,0,1,0,0,0,1,none
4,1,0,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,1,0,0,none


In [66]:
# Inserindo identificações
df_ohe['ID'] = dados['ID'].copy()

In [67]:
df_ohe['ID'].head()

0    0002-ORFBO
1    0003-MKNFE
2    0004-TLHLJ
3    0011-IGKFF
4    0013-EXCHZ
Name: ID, dtype: object

In [68]:
# Reinserindo dados acerca de pagamentos, posse e demais variáveis categóricas
df_ohe['CobrancasMensais'] = dados['CobrancasMensais'].copy()
df_ohe['CobrancasTotais'] = dados['CobrancasTotais'].copy()
df_ohe['Posse'] = dados['Posse'].copy()
df_ohe['Genero'] = dados['Genero'].copy()
df_ohe['Senior'] = dados['Senior'].copy()
df_ohe['Casado(a)'] = dados['Casado(a)'].copy()
df_ohe['Dependentes'] = dados['Dependentes'].copy()
df_ohe['CobrancasemPapel'] = dados['CobrancasemPapel'].copy()
df_ohe['Evasao'] = dados['Evasao'].copy()

In [69]:
# Analisando coposição do nosso Data Frame
df_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 41 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   MultiplasLinhas_Nao                            7043 non-null   uint8  
 1   MultiplasLinhas_Sem servico telefonico         7043 non-null   uint8  
 2   MultiplasLinhas_Sim                            7043 non-null   uint8  
 3   ServicodeInternet_DSL                          7043 non-null   uint8  
 4   ServicodeInternet_Fibra optica                 7043 non-null   uint8  
 5   ServicodeInternet_Nao                          7043 non-null   uint8  
 6   SegurancaOnline_Nao                            7043 non-null   uint8  
 7   SegurancaOnline_Sem servico de Internet        7043 non-null   uint8  
 8   SegurancaOnline_Sim                            7043 non-null   uint8  
 9   BackupOnline_Nao                               7043 

In [70]:
#Construção de dicionario para alterar características binárias
binario = {'Sim': 1, 'Nao':0, 'Homem': 0, 'Mulher': 1}

In [71]:
df_ohe = df_ohe.replace(binario)
df_ohe.head()

,MultiplasLinhas_Nao,MultiplasLinhas_Sem servico telefonico,MultiplasLinhas_Sim,ServicodeInternet_DSL,ServicodeInternet_Fibra optica,ServicodeInternet_Nao,SegurancaOnline_Nao,SegurancaOnline_Sem servico de Internet,SegurancaOnline_Sim,BackupOnline_Nao,...,ID,CobrancasMensais,CobrancasTotais,Posse,Genero,Senior,Casado(a),Dependentes,CobrancasemPapel,Evasao
0,1,0,0,1,0,0,1,0,0,0,...,0002-ORFBO,65.6,593.30,9,1,0,1,1,1,0
1,0,0,1,1,0,0,1,0,0,1,...,0003-MKNFE,59.9,542.40,9,0,0,0,0,0,0
2,1,0,0,0,1,0,1,0,0,1,...,0004-TLHLJ,73.9,280.85,4,0,0,0,0,1,1
3,1,0,0,0,1,0,1,0,0,0,...,0011-IGKFF,98.0,1237.85,13,0,1,1,0,1,1
4,1,0,0,0,1,0,1,0,0,1,...,0013-EXCHZ,83.9,267.40,3,1,1,1,0,1,1


In [72]:
#Avaliando quais categorias serão analisadas
df_ohe.columns

Index(['MultiplasLinhas_Nao', 'MultiplasLinhas_Sem servico telefonico',
       'MultiplasLinhas_Sim', 'ServicodeInternet_DSL',
       'ServicodeInternet_Fibra optica', 'ServicodeInternet_Nao',
       'SegurancaOnline_Nao', 'SegurancaOnline_Sem servico de Internet',
       'SegurancaOnline_Sim', 'BackupOnline_Nao',
       'BackupOnline_Sem servico de Internet', 'BackupOnline_Sim',
       'ProtecaodeDispositivo_Nao',
       'ProtecaodeDispositivo_Sem servico de Internet',
       'ProtecaodeDispositivo_Sim', 'SuporteTecnico_Nao',
       'SuporteTecnico_Sem servico de Internet', 'SuporteTecnico_Sim',
       'StreamingTV_Nao', 'StreamingTV_Sem servico de Internet',
       'StreamingTV_Sim', 'StreamingFilmes_Nao',
       'StreamingFilmes_Sem servico de Internet', 'StreamingFilmes_Sim',
       'Contrato_Anual', 'Contrato_Dois anos', 'Contrato_Mensal',
       'MetododePagamento_Cartao de credito',
       'MetododePagamento_Correspondencia',
       'MetododePagamento_Debito automatico', 'Metodo

In [73]:
features_ml_test = ['MultiplasLinhas_Nao', 'MultiplasLinhas_Sem servico telefonico',
       'MultiplasLinhas_Sim', 'ServicodeInternet_DSL',
       'ServicodeInternet_Fibra optica', 'ServicodeInternet_Nao',
       'SegurancaOnline_Nao', 'SegurancaOnline_Sem servico de Internet',
       'SegurancaOnline_Sim', 'BackupOnline_Nao',
       'BackupOnline_Sem servico de Internet', 'BackupOnline_Sim',
       'ProtecaodeDispositivo_Nao',
       'ProtecaodeDispositivo_Sem servico de Internet',
       'ProtecaodeDispositivo_Sim', 'SuporteTecnico_Nao',
       'SuporteTecnico_Sem servico de Internet', 'SuporteTecnico_Sim',
       'StreamingTV_Nao', 'StreamingTV_Sem servico de Internet',
       'StreamingTV_Sim', 'StreamingFilmes_Nao',
       'StreamingFilmes_Sem servico de Internet', 'StreamingFilmes_Sim',
       'Contrato_Anual', 'Contrato_Dois anos', 'Contrato_Mensal',
       'MetododePagamento_Cartao de credito',
       'MetododePagamento_Correspondencia',
       'MetododePagamento_Debito automatico', 'MetododePagamento_Eletronico',
     'CobrancasMensais', 'CobrancasTotais', 'Posse',
       'Genero', 'Senior', 'Casado(a)', 'Dependentes', 'CobrancasemPapel', 'Evasao']

In [74]:
# Atribuindo variáveis de análise e determinando as dimensões
X = df_ohe[features_ml_test].iloc[:, :-1].values
y = df_ohe[features_ml_test].iloc[:,-1].values

print(X.shape, y.shape)

(7043, 39) (7043,)


In [75]:
#Teste F para verificar impacto de característica univariada
[f_stat, f_p_value] = f_classif(X, y)
f_test_df = pd.DataFrame({'Feature': features_ml_test[:-1], 'F statistic': f_stat, 'p value': f_p_value})

f_test_df.sort_values('p value')

,Feature,F statistic,p value
26,Contrato_Mensal,1382.340697,1.991701e-276
33,Posse,997.268010,7.999058e-205
6,SegurancaOnline_Nao,936.566070,3.197002e-193
15,SuporteTecnico_Nao,903.785170,6.413692e-187
4,ServicodeInternet_Fibra optica,738.046042,1.200784e-154
25,Contrato_Dois anos,707.919254,1.048134e-148
30,MetododePagamento_Eletronico,706.195028,2.296890e-148
9,BackupOnline_Nao,544.868484,3.865357e-116
12,ProtecaodeDispositivo_Nao,479.401833,7.329625e-103
13,ProtecaodeDispositivo_Sem servico de Internet,385.698588,1.200504e-83


As características acima estão ordenadas de acordo com o seu fator de impacto na decisão dos clientes se manterem ou não com os serviços da Alura Voz. Sendo os primerios mais impactantes se comparados com os demais. Faremos agora a divisão dos dados a fim de se treinar e predizer os modelos com base nessa informação.

In [76]:
#Dividindo dados para teste e predição

X_train, X_test, y_train, y_test = train_test_split(df_ohe['Contrato_Mensal'].values.reshape(-1,1), df_ohe['Evasao'].values, test_size=0.2, random_state=24)


In [77]:
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

LogisticRegression()

In [78]:
y_pred_lr = lr_model.predict(X_test)

In [79]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [80]:
rf_model = RandomForestClassifier()

In [81]:
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [82]:
y_pred_rf = rf_model.predict(X_test)

In [83]:
y_pred_rf

array([0, 0, 0, ..., 0, 0, 0])